In [4]:
using CSV
using DataFrames
using Optim
using Distributions
using LinearAlgebra   
using Plots
using Glob
using Printf

In [ ]:
particle_data = CSV.read("/Users/carlitos/Desktop/TAMBO/plots/reference_plot/data/particles.csv", DataFrame)

In [ ]:
#= function cartesian_distance(x1, y1, z1, x2, y2, z2)
    return sqrt((x2 - x1)^2 + (y2 - y1)^2 + (z2 - z1)^2)
end

x1, y1, z1 = 5.82690291372798, 2.651239329640523, -0.7696777502523529 # Coordinates of the first point
x2, y2, z2 = 2.3913633278814264, 1.8136136167654868, −0.5130112708971727  # Coordinates of the second point

# Calculate the distance
distance = cartesian_distance(x1, y1, z1, x2, y2, z2)
println("The distance between the points is: $distance")

# Calculate t0
c = 299792 # km/s
t0 = distance/c
println("The value for t0 is: $t0") =#

In [ ]:
normal_vector = [0.45217398524533681 -0.3661629880519191 0.81330397346148509]
x_axis = [0.0 -0.91184756344828699 -0.41052895273466672]
y_axis = [0.89192975455881607 0.18563051261662877 -0.41231374670066206]

rotation_matrix = vcat(x_axis, y_axis, normal_vector)

coords = hcat(particle_data.x, particle_data.y, particle_data.z)

rotated_positions = inv(rotation_matrix) * transpose(coords)

transposed_positions = transpose(rotated_positions)

df_rotated_positions = DataFrame(transposed_positions, [:x, :y, :z])
df_kinetic_energy = DataFrame(kinetic_energy = particle_data.kinetic_energy)

df_rotated_positions[!, :kinetic_energy] = df_kinetic_energy[!, :kinetic_energy]

println(df_rotated_positions)

sqrt_total_energy = sum(sqrt.(particle_data.kinetic_energy))

weights = sqrt.(df_rotated_positions.kinetic_energy) ./ sqrt_total_energy

df_rotated_positions[!, :weighted_x] = df_rotated_positions.x .* weights
df_rotated_positions[!, :weighted_y] = df_rotated_positions.y .* weights
df_rotated_positions[!, :weighted_z] = df_rotated_positions.z .* weights

weighted_x = sum(df_rotated_positions[!, :weighted_x])
weighted_y = sum(df_rotated_positions[!, :weighted_y])
weighted_z = sum(df_rotated_positions[!, :weighted_z])

println(weighted_x)
println(weighted_y)
println(weighted_z)

println(df_rotated_positions)

In [ ]:
# CSV.write("rotated_positions.csv", df_rotated_positions)

In [ ]:
a = 4.823e-4 / 1e9  # in s/m^2
b = 19.41 / 1e9     # in seconds (s)
sigma = 83.5  # in meters (m)
c = 299792458  # Speed of light in meters per second (m/s)
#t0 = 2.8612842910494837e-5  # Initial time in seconds using intecept 
t0 = 1.1826465565367665e-5 # Initial time in seconds using mean position of particles 

initial_position = [5826.90291372798, 2651.239329640523, -769.6777502523529]
pos = [-1904.2410053751477, -753.5013903260498, 719.4652218340803]
core = [2391.3633278814264, 1813.6136167654868, -513.0112708971727]

zenith_rad = deg2rad(180 - 99.99732787664736)
azimuth_rad = deg2rad(203.76835088101396)
direction_vector = (
    xdir = sin(zenith_rad) * cos(azimuth_rad),
    ydir = sin(zenith_rad) * sin(azimuth_rad),
    zdir = cos(zenith_rad)
)

# println("Direction vector: $direction_vector")

function solved_R(pos, core, direction_vector)
    if all(pos .== values(core))
        return 0 
    end 
    result = norm((pos .- values(core)) * sin(acos(dot(normalize((pos .- values(core))), direction_vector))))
    # println("R calculation: result = $result")
    return result
end

R = solved_R(pos, core, direction_vector)

function delta_t(a, b, sigma, R)
    result = a * R^2 + b * (1 - exp(-R^2 / (2 * sigma^2)))
    # println("Δt calculation: result = $result")
    return result
end

delta_t_val = delta_t(a , b, sigma, R)

function expected_signal_time(t0, pos, core, a, b, sigma, direction_vector)
    c = 299792458
    R = solved_R(pos, core, direction_vector) 
    delta_t_val = delta_t(a, b, sigma, R)
    dot_product = dot((pos .- values(core)), direction_vector)
    # println("Intermediate values: pos = $pos, core = $core, direction_vector = $direction_vector, dot_product = $dot_product")
    signal_time = t0 + (dot_product / c) + delta_t_val
    # println("Signal time calculation: t0 = $t0, dot_product/c = $(dot_product / c), Δt = $delta_t_val, signal_time = $signal_time")
    return signal_time
end

expected_time = []
for i in eachcol(rotated_positions)
    pos = i
    signal_time = expected_signal_time(t0, pos, core, a, b, sigma, direction_vector)
    push!(expected_time, signal_time)
end

# println("expected_time = $expected_time")


In [ ]:
df_rotated_positions[!, :expected_time] = expected_time

df_rotated_positions[!, :expected_time] = convert(Vector{Float64}, df_rotated_positions[!, :expected_time])

println(df_rotated_positions)

In [ ]:
scatter(rotated_positions[1, :], rotated_positions[2, :], marker_z=log10.(particle_data.time), color=:thermal, xlabel="X Position (m)", ylabel="Y Position (m)", title="True Particle Arrival Times (log_10 seconds)")
# savefig("/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/true_times.png")

In [ ]:
scatter(rotated_positions[1, :], rotated_positions[2, :], marker_z=log10.(abs.(particle_data.time - expected_time)), color=:thermal, xlabel="X Position (m)", ylabel="Y Position (m)", title="Expected Particle Times (log_10 seconds)")
# savefig("/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/expected_times.png")

In [ ]:
neg_indices = expected_time .< 0
scatter(rotated_positions[1, :][neg_indices], rotated_positions[2, :][neg_indices], marker_z=log10.(abs.(particle_data.time[neg_indices] .- expected_time[neg_indices])), color=:thermal, xlabel="X Position (m)", ylabel="Y Position (m)", title="Expected Particle Times (log_10 seconds)")
# savefig("/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/negative_expected_times.png")

In [ ]:
#= 
directory = "/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/data"
println("Directory: $directory")

sim_test = ["sim_test_?????_?", "sim_test_????_?", "sim_test_???_?", "sim_test_??_?"]

dirs = glob("sim_test*", directory)
println(dirs)

sim_test_dirs = []
for pattern in sim_test
    dirs = glob(pattern, directory)
    println("Pattern: $pattern found directories: $(length(dirs))")
    append!(sim_test_dirs, dirs)
end

config_files = []
summary_files = []

for dir in sim_test_dirs
    println("Checking directory: $dir")
    
    config = glob("config.yaml", dir)
    if !isempty(config)
        println("Found config file: $config[1]")
        append!(config_files, config[1])
    else
        println("No config file found in $dir")
    end
    
    particles_obs_final_dir = glob("particles_obs_final", dir)
    if !isempty(particles_obs_final_dir)
        particles_obs_final_path = particles_obs_final_dir[1]
        println("Found particles_obs_final directory: $particles_obs_final_path")
        
        summary = glob("summary*.yaml", particles_obs_final_path)

        # Logging found files
        if isempty(summary)
            println("No summary files found in $particles_obs_final_path")
        else
            println("Found summary files: ", join(summary, ", "))
        end
        
        # Collecting found files
        append!(summary_files, summary)
    else
        println("No particles_obs_final directory found in $dir")
    end
end

# Print the found files
@printf "Config Files:\n%s\n" join(config_files, "\n")
@printf "Summary Files:\n%s\n" join(summary_files, "\n")
 =#

In [24]:
using Glob
using YAML
using Plots

directory = "/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/data"
# println("Directory: $directory")

configs = glob("sim_test*/config.yaml", directory)
# println(configs)

function read_yaml(file_path)
    open(file_path) do file
        return YAML.load(file)
    end
end

function extract_zenith_azimuth(args)
    zenith_azimuth = match(r"--zenith\s+(\S+)\s+--azimuth\s+(\S+)", args)
    if zenith_azimuth !== nothing
        zenith = parse(Float64, zenith_azimuth.captures[1])
        azimuth = parse(Float64, zenith_azimuth.captures[2])
        return zenith, azimuth
    else
        return nothing, nothing
    end
end

zeniths = Float64[]
azimuths = Float64[]

for config in configs
    # println("\nReading config file: $config")
    config_data = read_yaml(config)
    args = config_data["args"]
    zenith, azimuth = extract_zenith_azimuth(args)
    new_zenith = π - zenith
    push!(zeniths, rad2deg(new_zenith))
    push!(azimuths, rad2deg(azimuth))
    println("Zenith: $(rad2deg(new_zenith)), Azimuth: $(rad2deg(azimuth))")
end


Zenith: 101.28220534116271, Azimuth: 194.31881869233737
Zenith: 101.2815896294578, Azimuth: 194.3226806009826
Zenith: 106.53285968627058, Azimuth: 176.5949815695885
Zenith: 106.53233959455619, Azimuth: 176.59456275104048
Zenith: 98.80246943389828, Azimuth: 104.56409928740825
Zenith: 98.80218900310715, Azimuth: 104.56344488941718
Zenith: 117.62360525437872, Azimuth: 139.48566041100702
Zenith: 117.68166403167305, Azimuth: 139.32334475149634
Zenith: 117.5639282854073, Azimuth: 139.42580728693807
Zenith: 117.65985925161527, Azimuth: 139.3893234002133
Zenith: 93.83642433070591, Azimuth: 184.427141477241
Zenith: 93.8362897555101, Azimuth: 184.42712518038527
Zenith: 93.8364069254144, Azimuth: 184.42707636346913
Zenith: 116.23563517765776, Azimuth: 154.5436239242679
Zenith: 107.1907764509, Azimuth: 183.87895004403126
Zenith: 106.90402913194458, Azimuth: 184.5906037160942
Zenith: 94.58505366290336, Azimuth: 159.87894778683702
Zenith: 89.04835574053816, Azimuth: 207.8679696015989
Zenith: 102.124

In [27]:
histogram(zeniths, bins=33, title="Zenith Histogram", xlabel="Zenith (degrees)", ylabel="Frequency")
# savefig("/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/zenith_histogram.png")  


"/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/zenith_histogram.png"

In [28]:
histogram(azimuths, bins=33, title="Azimuth Histogram", xlabel="Azimuth (degrees)", ylabel="Frequency")
# savefig("/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/azimuth_histogram.png") 

"/Users/carlitos/Desktop/TAMBO/plots/air_shower_reconstruction/output/azimuth_histogram.png"

In [ ]:
function meshgrid(x, y)
    X = repeat(x, 1, length(y))
    Y = repeat(y', length(x), 1)
    return X, Y
end

In [ ]:
expected_time = []
x_i = -10000:100:10000
y_i = -10000:100:10000
x_co = [x for x in x_i, y in y_i]
y_co = [y for x in x_i, y in y_i]
normal_vector = [0.45217398524533681 -0.3661629880519191 0.81330397346148509]
z = -(normal_vector[1]*x_co -normal_vector[2]*y_co)/normal_vector[3]

mesh_pos = (x_co,y_co,z)
mesh_pos[0]
for i in eachcol(rotated_positions)
    pos = i
    signal_time = expected_signal_time(t0, pos, core, a, b, sigma, direction_vector)
    push!(expected_time, signal_time)
end

In [ ]:
function optimization(params) 
    zenith_rad = deg2rad(180 - 99.99732787664736)
    azimuth_rad = deg2rad(203.76835088101396)
    direction_vector = [
        sin(zenith_rad) * cos(azimuth_rad),
        sin(zenith_rad) * sin(azimuth_rad),
        cos(zenith_rad)
    ]
    t0 = 1.1826465565367665e-5
    core = [2391.3633278814264, 1813.6136167654868, -513.0112708971727]  
    a, b, sigma = params
    t_true = particle_data.time
    t_expected = []
    for pos in eachcol(rotated_positions)
        push!(t_expected, expected_signal_time(t0, pos, core, a, b, sigma, direction_vector))
    end
    return sum((t_expected .- t_true) .^ 2) 
end

initial_params = [a, b, sigma]
result = optimize(optimization, initial_params, LBFGS())
optimized_params = Optim.minimizer(result)

println("Optimized parameters: $optimized_params")


In [ ]:
histogram(abs.((particle_data.time .- t0) - expected_time)./(particle_data.time .- t0))

In [ ]:
histogram(particle_data.time .- t0)

In [ ]:
time_negatives = 0

for time in expected_time
    if time < 0
        println(time)
        time_negatives += 1
    end
end

println("# of negatives: $time_negatives")

#= R_negatives = 0

for row in eachrow(df_rotated_positions)
    pos = [row[:x], row[:y], row[:z]]
    R = solved_R(pos, core, direction_vector)
    if R < 0
        println("Negative R value detected: R = $R, pos = $pos, core = $core, direction_vector = $direction_vector")
        R_negatives += 1
    end
end
 =#
 
#= delta_negatives = 0

for row in eachrow(df_rotated_positions)
    pos = [row[:x], row[:y], row[:z]]
    R = solved_R(pos, core, direction_vector)
    delta_t_val = delta_t(a, b, sigma, R)
    if delta_t_val < 0
        println("Negative Δt value detected: Δt = $delta_t_val, R = $R, pos = $pos, core = $core, direction_vector = $direction_vector")
        delta_negatives += 1
    end
end =#
